# Save recommender


In [64]:
import pandas as pd
import os, sys

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing import image
from typing import Union, Optional, Tuple, Dict, Text

from PIL import Image
import matplotlib.pyplot as plt 
from time import time
import numpy as np

sys.path.append('../src')

import settings
import utils
from utils import make_fpath_from_id
from preprocess_images import resize_image
from utils_tf import ImageNormalizer, parse_image_func
from create_recommender import test_an_id, create_image_query_from_fpath


In [65]:
tfrs.__version__

'v0.6.0'

In [66]:
tf.__version__

'2.6.0'


# load features dataset

In [103]:
fpath_feat = "../data/processed/ethz/features/features_prod.csv"
fpath_feat = settings.interim_features_fpath
df_feat = pd.read_csv(fpath_feat, header=None, index_col=0)

In [104]:
df_feat.head()

,1,2,3,4,5,6,7,8,9,10,...,503,504,505,506,507,508,509,510,511,512
0,,,,,,,,,,,,,,,,,,,,,
45,0.223475,-0.557490,0.638113,-0.512727,-0.017547,-0.557490,-0.557490,0.669734,-0.557490,-0.557490,...,2.072416,0.254421,0.855285,-0.153446,-0.557490,-0.042730,0.497180,-0.557490,1.194554,-0.557490
14,0.218186,-0.622638,1.761528,-0.544548,-0.317433,-0.622638,-0.622638,-0.622638,-0.622638,0.046054,...,2.550667,0.671487,1.421005,0.598957,-0.622638,0.515799,0.205608,-0.622638,0.854591,-0.622638
69,0.508393,-0.662697,0.048061,-0.662697,0.868822,-0.662697,-0.079438,-0.477359,-0.662697,-0.662697,...,1.741087,-0.393043,-0.112213,1.487349,-0.662697,1.292760,0.778660,-0.662697,0.264572,-0.662697
38,0.491271,-0.584909,1.954061,-0.584909,-0.015819,-0.584909,-0.313723,0.671707,-0.323853,-0.584909,...,2.201645,0.174490,1.663462,-0.320516,-0.584909,0.497191,0.185504,-0.584909,0.469529,-0.584909
68,1.712878,-0.839283,2.311375,-0.690416,-0.480790,-0.839283,1.027839,-0.116085,-0.839283,0.447241,...,1.534335,0.653519,0.307561,-0.839283,-0.404575,0.405402,1.068561,-0.839283,-0.014958,-0.839283


In [105]:
# should eventually be a call to the feature store with all features

# fpath_meta = "../data/processed/ethz/met/features_prod.csv"
# fpath_feat = "../data/processed/ethz/features/features_vgg16_imagenet.csv"
fpath_meta = settings.processed_metadata_dir + "/metadata.csv"
df_meta = pd.read_csv(fpath_meta,index_col=0)
# df_feat.index = df_feat.reset_index()[0].str.rsplit('/', n=1, expand=True)[1].str.split('.', expand=True)[0]

In [106]:
features = df_feat.values
identifiers = df_feat.index

### Load feature extraction model

In [107]:
!ls ../models

feature_extraction  retrieval  retrieval_exclusion


In [108]:
# load the trained feature extraction model
# fpath_model = "../models/feature_extraction/202109081606"
fpath_model = "../models/feature_extraction/2"
ftx_model = tf.keras.models.load_model(fpath_model)

In [109]:
fpath_model = "../models/retrieval/2"
retrieval_model = tf.keras.models.load_model(fpath_model)

In [110]:
fpath_model = "../models/retrieval_exclusion/2"
retrieval_exclusion_model = tf.keras.models.load_model(fpath_model)


## One Hot Encoding

In [111]:
from sklearn.preprocessing import OneHotEncoder

In [112]:
one_h = OneHotEncoder()

In [113]:
df_meta.head()

,record_id,image_url,inventory_number,person,date,title,classification,material_technique,institution_isil,record_url,image_licence,image_fpath,year_min,year_max,relationship_type_id,classification_id,material_technique_id,institution_isil_id,created_date
id,,,,,,,,,,,,,,,,,,,
0,14715,https://e-gs.ethz.ch/eMP/eMuseumPlus?service=I...,D 001022,"Klee, Paul (1879 - 1940), Künstler",Enstehung des Druckträgers: 7.1903,Jungfrau im Baum,Druckgraphik,"Zink, Radierung",Graphische Sammlung ETH Zürich (CH-000511-9),https://e-gs.ethz.ch/eMP/eMuseumPlus?service=E...,Public Domain Mark 1.0,../data/processed/ethz/images/0/0.jpeg,1903,1903,NaN,1,"[18, 14]",1,2021-09-20T15:58:48+01:00
1,249848,https://e-gs.ethz.ch/eMP/eMuseumPlus?service=I...,D 026149,"Anonym (Datierung unbekannt), Künstler/in",Enstehung des Druckträgers: 1650 - 1700 [zweit...,Schwarzornament mit Blüten,Druckgraphik,"Radierung, Papier vergé",Graphische Sammlung ETH Zürich (CH-000511-9),https://e-gs.ethz.ch/eMP/eMuseumPlus?service=E...,Public Domain Mark 1.0,../data/processed/ethz/images/0/1.jpeg,1650,1700,NaN,1,"[14, 13]",1,2021-09-20T15:58:48+01:00
2,28839,https://e-gs.ethz.ch/eMP/eMuseumPlus?service=I...,D 006496,"Uden, Lucas van (1595 - 1672)",Enstehung des Druckträgers: 1610 - 1672,"Landschaft mit Fluss, Brücke und Reitern, Blat...",Druckgraphik,"Radierung, Kupferstich",Graphische Sammlung ETH Zürich (CH-000511-9),https://e-gs.ethz.ch/eMP/eMuseumPlus?service=E...,Public Domain Mark 1.0,../data/processed/ethz/images/0/2.jpeg,1610,1672,NaN,1,"[14, 10]",1,2021-09-20T15:58:48+01:00
3,21883,https://e-gs.ethz.ch/eMP/eMuseumPlus?service=I...,D 005593,"Bol, Ferdinand (1616 um - um 1680)",Enstehung des Druckträgers: 1642,Porträt eines jungen Mannes mit gefiedertem Ba...,Druckgraphik,"Kaltnadel, Radierung",Graphische Sammlung ETH Zürich (CH-000511-9),https://e-gs.ethz.ch/eMP/eMuseumPlus?service=E...,Public Domain Mark 1.0,../data/processed/ethz/images/0/3.jpeg,1642,1642,NaN,1,"[9, 14]",1,2021-09-20T15:58:48+01:00
4,28305,https://e-gs.ethz.ch/eMP/eMuseumPlus?service=I...,D 005288,"Rembrandt Harmensz. van Rijn (1606 - 1669), Kü...",Enstehung des Druckträgers: 1638,Selbstporträt mit Samthut mit Feder,Druckgraphik,"Radierung, Papier vergé",Graphische Sammlung ETH Zürich (CH-000511-9),https://e-gs.ethz.ch/eMP/eMuseumPlus?service=E...,Public Domain Mark 1.0,../data/processed/ethz/images/0/4.jpeg,1638,1638,[1],1,"[14, 13]",1,2021-09-20T15:58:48+01:00


In [114]:
one_h.fit(df_meta[['classification']].values)

OneHotEncoder()

In [115]:
one_hot_classes = one_h.transform(df_meta[['classification']].values)

In [116]:
one_hot_classes

<100x2 sparse matrix of type '<class 'numpy.float64'>'
	with 100 stored elements in Compressed Sparse Row format>

In [117]:
df_meta.columns

Index(['record_id', 'image_url', 'inventory_number', 'person', 'date', 'title',
       'classification', 'material_technique', 'institution_isil',
       'record_url', 'image_licence', 'image_fpath', 'year_min', 'year_max',
       'relationship_type_id', 'classification_id', 'material_technique_id',
       'institution_isil_id', 'created_date'],
      dtype='object')

In [118]:
ser

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
id,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
98,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [119]:
ser = df_meta['material_technique_id'].apply(eval)
ser = ser.explode()
ser = pd.get_dummies(ser)
one_hot_mat_tec = ser.groupby(by='id').sum()
one_hot_mat_tec

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
id,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
96,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
97,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0


In [120]:
one_hot_class = pd.get_dummies(df_meta['classification_id'])
one_hot_class

,1,2
id,,
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
95,1,0
96,1,0
97,1,0


# querying

In [121]:
# Padding exclusion input (not used)

# exclusions = tf.pad(exclusions, tf.constant([[0,0],[0, num_pad]]), constant_values=-1)

### Make a new model

In [122]:
### THIS IS THE ONE

In [123]:


class BruteForceExclude(tfrs.layers.factorized_top_k.BruteForce):
    """Brute force retrieval."""

    def __init__(self,
               query_model: Optional[tf.keras.Model] = None,
               k: int = 10,
               name: Optional[Text] = None):
        """Initializes the layer.
        Args:
          query_model: Optional Keras model for representing queries. If provided,
            will be used to transform raw features into query embeddings when
            querying the layer. If not provided, the layer will expect to be given
            query embeddings as inputs.
          k: Default k.
          name: Name of the layer.
        """

        super().__init__(query_model=query_model, k=k, name=name)


    def _take_along_axis(self,
                         arr: tf.Tensor, indices: tf.Tensor) -> tf.Tensor:
        """Partial TF implementation of numpy.take_along_axis.
        See
        https://numpy.org/doc/stable/reference/generated/numpy.take_along_axis.html
        for details.
        Args:
        arr: 2D matrix of source values.
        indices: 2D matrix of indices.
        Returns:
        2D matrix of values selected from the input.
        """

        row_indices = tf.tile(
          tf.expand_dims(tf.range(tf.shape(indices)[0]), 1),
          [1, tf.shape(indices)[1]])
        gather_indices = tf.concat(
          [tf.reshape(row_indices, (-1, 1)),
           tf.reshape(indices, (-1, 1))], axis=1)

        return tf.reshape(tf.gather_nd(arr, gather_indices), tf.shape(indices))


    def _exclude(self,
                 scores: tf.Tensor, 
                 identifiers: tf.Tensor, 
                 exclude: tf.Tensor,
                 k: int) -> Tuple[tf.Tensor, tf.Tensor]:
        """Removes a subset of candidates from top K candidates.
        For each row of inputs excludes those candidates whose identifiers match
        any of the identifiers present in the exclude matrix for that row.
        Args:
        scores: 2D matrix of candidate scores.
        identifiers: 2D matrix of candidate identifiers.
        exclude: 2D matrix of identifiers to exclude.
        k: Number of candidates to return.
        Returns:
        Tuple of (scores, indices) of candidates after exclusions.
        """

        idents = tf.expand_dims(identifiers, -1)
        exclude = tf.expand_dims(exclude, 1)

        isin = tf.math.reduce_any(tf.math.equal(idents, exclude), -1)

        # Set the scores of the excluded candidates to a very low value.
        adjusted_scores = (scores - tf.cast(isin, tf.float32) * 1.0e5)

        k = tf.math.minimum(k, tf.shape(scores)[1])

        _, indices = tf.math.top_k(adjusted_scores, k=k)

        return self._take_along_axis(scores,
                              indices),self._take_along_axis(identifiers, indices)

    @tf.function()
    def query_with_ragged_exclusions(
          self,
          queries: Union[tf.Tensor, Dict[Text, tf.Tensor]],
          exclusions: tf.Tensor,
          k: Optional[int] = None) -> Tuple[tf.Tensor, tf.Tensor]:
        """Query the index.
        Args:
          queries: Query features. If `query_model` was provided in the constructor,
            these can be raw query features that will be processed by the query
            model before performing retrieval. If `query_model` was not provided,
            these should be pre-computed query embeddings.
          exclusions: `[query_batch_size, num_to_exclude]` tensor of identifiers to
            be excluded from the top-k calculation. This is most commonly used to
            exclude previously seen candidates from retrieval. For example, if a
            user has already seen items with ids "42" and "43", you could set
            exclude to `[["42", "43"]]`.
          k: The number of candidates to retrieve. Defaults to constructor `k`
            parameter if not supplied.
        Returns:
          Tuple of (top candidate scores, top candidate identifiers).
        Raises:
          ValueError if `index` has not been called.
          ValueError if `queries` is not a tensor (after being passed through
            the query model).
        """

        # Ideally, `exclusions` would simply be an optional parameter to
        # `call`. However, Keras is unable to handle `call` signatures
        # that have more than one Tensor input parameter. The alternative
        # is to either pack all inputs into the first positional argument
        # (via tuples or dicts), or else have a separate method. We opt
        # for the second solution here. The ergonomics in either case aren't
        # great, but having two methods is simpler to explain.
        # See https://github.com/tensorflow/tensorflow/blob/v2.4.0/tensorflow/
        # python/keras/engine/base_layer.py#L942 for details of why Keras
        # puts us in this predicament.

        k = k if k is not None else self._k
        
#         num_exclusions = exclusions.bounding_shape()[1]
        num_exclusions = exclusions.shape[1]
#         print("num_exclusions: ", num_exclusions, "  , ", type(num_exclusions))
        adjusted_k = tf.add(tf.constant(k, dtype=tf.int32), tf.constant(num_exclusions, dtype=tf.int32))
        adjusted_k = tf.cast(adjusted_k, tf.int32)
        x, y = self.query_without_exclusions(queries=queries, k=adjusted_k)
#         exclusions = exclusions.to_tensor()
        
        return self._exclude(x, y, exclude=exclusions, k=k)
    
    @tf.function()
    def query_without_exclusions(
          self,
          queries: Union[tf.Tensor, Dict[Text, tf.Tensor]],
          k: Optional[int] = None,
        ) -> Tuple[tf.Tensor, tf.Tensor]:

        k = k if k is not None else self._k

        scores = self._compute_score(queries, self._candidates)
        values, indices = tf.math.top_k(scores, k=k)
        return values, tf.gather(self._identifiers, indices)
    
    @tf.function()
    def call(
          self,
          queries: Union[tf.Tensor, Dict[Text, tf.Tensor]],
          exclusions: Optional[tf.RaggedTensor]= None,
          k: Optional[int] = None,
        ) -> Tuple[tf.Tensor, tf.Tensor]:

        k = k if k is not None else self._k
        
        if self._candidates is None:
            raise ValueError("The `index` method must be called first to "
                           "create the retrieval index.")

        if self.query_model is not None:
            queries = self.query_model(queries)
        
        if exclusions is None:
            return self.query_without_exclusions(queries, k)

        else:

            return self.query_with_ragged_exclusions(queries,
                        exclusions,
                        k)
       


In [124]:
retrieval_model = BruteForceExclude(ftx_model)

In [125]:
candidates = tf.constant(features, dtype=tf.float32)
identifiers = tf.constant(identifiers, dtype=tf.int32)

retrieval_model.index(candidates, identifiers)

In [126]:

x = tf.expand_dims(candidates[0], 0)
x.shape

TensorShape([1, 512])

In [127]:
# overall function inputs
img_fpath = make_fpath_from_id(0, image_fldr=settings.processed_image_dir)
query_img = create_image_query_from_fpath(img_fpath)
# query = x
exclusions_list = [0,40, 64, 76, 83, 14, 56, 21, 74,2, ]
exclusions = tf.ragged.constant(exclusions_list, dtype=tf.int32)
exclusions.shape

TensorShape([10])

In [128]:
exclusions = tf.ragged.constant(exclusions_list, dtype=tf.int32)
exclusions = tf.expand_dims(exclusions_list,0)

In [129]:
retrieval_model(query_img, exclusions, k=10)

(<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[441.10284, 440.09918, 439.2088 , 439.15936, 437.8673 , 437.07272,
         436.50256, 436.32697, 436.24332, 436.1353 ]], dtype=float32)>,
 <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[97, 90, 82, 13, 78,  3, 58, 53, 33, 12]], dtype=int32)>)

In [130]:
retrieval_model(query_img, k=10)

(<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[510.99402, 450.3332 , 448.35434, 447.97958, 447.58505, 446.63544,
         443.932  , 442.3553 , 441.83124, 441.6549 ]], dtype=float32)>,
 <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[ 0, 40, 64, 76, 83, 14, 56, 21, 74,  2]], dtype=int32)>)

In [131]:
model_fpath = "../models/retrieval/3"



tf.saved_model.save(retrieval_model,
                   model_fpath,
                    options=tf.saved_model.SaveOptions(namespace_whitelist=["Scann"])
                   )

INFO:tensorflow:Assets written to: ../models/retrieval/3/assets


INFO:tensorflow:Assets written to: ../models/retrieval/3/assets


In [133]:
reloaded_model = tf.saved_model.load(model_fpath)

In [134]:
reloaded_model(query_img, exclusions, k=10)

ValueError: Could not find matching function to call loaded from the SavedModel. Got:
  Positional arguments (4 total):
    * Tensor("queries:0", shape=(1, 224, 224, 3), dtype=float64)
    * Tensor("exclusions:0", shape=(1, 10), dtype=int32)
    * 10
    * False
  Keyword arguments: {}

Expected these arguments to match one of the following 2 option(s):

Option 1:
  Positional arguments (4 total):
    * TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='queries')
    * TensorSpec(shape=(None, 10), dtype=tf.int32, name='exclusions')
    * None
    * False
  Keyword arguments: {}

Option 2:
  Positional arguments (4 total):
    * TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='queries')
    * TensorSpec(shape=(None, 10), dtype=tf.int32, name='exclusions')
    * None
    * True
  Keyword arguments: {}